In [5]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
import scipy.stats as sp

In [6]:
bg_level = 50 # ppb
measurements = np.array([30, 74, 48, 84, 78])
n = 5 
df = 4

In [80]:
# (a)

mean = np.mean(measurements)
sd = np.std(measurements, ddof=1)
sderr = sd / m.sqrt(n)
print(f"Mean: {mean}\nSD: {sd}\nStandard Error: {sderr}")

Mean: 62.8
SD: 22.91724241700995
Standard Error: 10.248902380255165


**# (b)**

Null hypothesis: The average concentration of ozone in unpolluted tropospheric air has not elevated above the background.
This is a one-tailed test because we are only looking at values *above*.

In [34]:
# (c)

t = (mean - bg_level) / sderr
p = 1 - sp.t.cdf(t, df=df)
p
print(f"No, we can't reject the null at a statistical significance level of 0.05 because we got a p-value of {p}. {t}")

No, we can't reject the null at a statistical significance level of 0.05 because we got a p-value of 0.1398984745088393. 1.2489142276015432


**# (d)**

The p-value we got in (c) means that we can reject the null with a significance value of 0.12. Conceptually speaking, alpha is the probability that we would see results as extreme as our sample, given that the null hypothesis is true (i.e. given that, in reality, pollution has not rasied ozone concentrations above the background).

In [35]:
# (e)

t_crit = sp.t.ppf(0.95, df=df)
diff = bg_level + (t_crit * sderr)
lsd = t_crit*sderr
pl = 1 - sp.t.cdf(lsd/sderr, df=df)
diff, lsd, pl, t_crit

# The LSD is 21.8

(71.84908955157681, 21.849089551576814, 0.05000000028058349, 2.13184678133629)

In [44]:
# (f)

our_difference = mean - bg_level

def get_t_crit(n):
    return sp.t.ppf(0.95, df=n-1)

def LSN_wrap(t_crit, diff, sd):
    return ((t_crit * sd) / abs(diff)) ** 2
    
def LSN(diff=our_difference, sd=sd, guess_n=10):
    t_crit = get_t_crit(guess_n)
    print(f"Guess n: {m.ceil(guess_n)}\tTcrit: {t_crit}\tLSN: {LSN_wrap(t_crit, diff, sd)}")
    if m.ceil(abs(LSN_wrap(t_crit, diff, sd))) == m.ceil(abs(guess_n)):
        return m.ceil(guess_n)
    guess_LSN = LSN_wrap(t_crit, diff, sd)
    
    if m.ceil(abs(guess_LSN)) == m.ceil(abs(LSN(diff, sd, guess_LSN))):
        return m.ceil(guess_LSN)
    return LSN(diff, sd, guess_LSN)

lsn = LSN()
print(f"The LSN that is required to make our measured deviation from background statistically significant at alpha=0.05 is {lsn}.")

Guess n: 10	Tcrit: 1.8331129326536335	LSN: 10.771674488112337
Guess n: 11	Tcrit: 1.8167680140998432	LSN: 10.580439991198507
The LSN that is required to make our measured deviation from background statistically significant at alpha=0.05 is 11.


In [46]:
# (g)

elevated_n = 5
elevated_df = elevated_n - 1
dev = 25

el_t = dev / (sd / m.sqrt(elevated_n))
el_t
p = 1 - sp.t.cdf(el_t, df=elevated_df)

t_beta = el_t - sp.t.ppf(1-.05, df=elevated_df)
beta = 1 - sp.t.cdf(t_beta, df=elevated_df)
beta
power = 1 - beta
power

print(f"The approximate power my test is {power}, the value of beta is {beta}, and beta is the probability of a false negative.")
el_t, t_beta

The approximate power my test is 0.6130742823406763, the value of beta is 0.3869257176593237, and beta is the probability of a false negative.


(2.4392856007842645, 0.3074388194479747)

In [13]:
el_t, t_beta

(2.727204209945015, 0.5953574286087253)

In [81]:
# (h)

def MDD(n, sd=sd):
    mdd = (sp.t.ppf(.95, n-1) + sp.t.ppf(.90, n-1)) * (sd / m.sqrt(n))
    # print(f"n={n}\tStderr={sd/m.sqrt(n)}\tMDD={mdd}\tt_alpha={sp.t.ppf(.95, n-1)}\tt_beta={sp.t.ppf(.9, n-1)}")
    return (sp.t.ppf(.95, n-1) + sp.t.ppf(.90, n-1)) * (sd / m.sqrt(n))

min_dd = MDD(5)
min_dd

print(f"The MDD is {min_dd}.")

# (i)

n_for_10_mdd = 2
while min_dd > 10:
    min_dd = MDD(n_for_10_mdd)
    n_for_10_mdd += 1

print(f"We should plan on a sample size of at least {n_for_10_mdd - 1} to to detect deviations of 10 ppb with a reliability of 90%.")

def get_t_crit(n):
    return sp.t.ppf(0.95, df=n-1)
    
def MDD_10(guess_n=20):
    # print(f"Guess n: {m.ceil(guess_n)}\tTcrit: {t_crit}\tLSN: {LSN_wrap(t_crit, diff, sd)}")
    if MDD(guess_n) <= 10:
        return m.ceil(guess_n)
    guess_MDD = MDD(guess_n)
    next_n = guess_n + (10/guess_MDD)
    print(next_n)
    if guess_MDD <= 10:
        return guess_MDD
    return MDD_10(next_n)
MDD_10()

The MDD is 37.56277097965761.
We should plan on a sample size of at least 47 to to detect deviations of 10 ppb with a reliability of 90%.
20.638376578951394
21.287801478831675
21.948274689284315
22.61979626289932
23.302366308212886
23.99598498342351
24.70065249090564
25.41636907187754
26.143135000310647
26.88095058360654
27.62981615411284
28.389732067522722
29.16069870259996
29.942716450838667
30.735785721473476
31.53990693470648
32.355080526493566
33.18130693983476
34.018586628770755
34.86692004722688
35.72630765783285
36.59674992729601
37.47824732538229
38.37080032400218
39.27440939639143
40.189075016377224
41.114797657721695
42.05157779353514
42.99941589575234
43.95831243466594
44.92826787851128
45.909282693097914
46.90135734148328


47